## The Ultimate Number Strategy Game

In [ ]:
import sys
import time
from colorama import Fore, Style, init
import winsound  # For Windows sound effects
from typing import List, Dict, Tuple
import json
from datetime import datetime

# Initialize colorama
init()

def play_sound(type: str):
    """Play different sound effects based on event type"""
    try:
        if type == "welcome":
            winsound.Beep(440, 200)
            winsound.Beep(523, 200)
        elif type == "eliminate":
            winsound.Beep(200, 500)
        elif type == "winner":
            winsound.Beep(440, 200)
            winsound.Beep(659, 200)
    except:
        pass  # Silently fail if sound doesn't work

def save_game_history(history: List[Dict]):
    """Save game replay to a JSON file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"game_replay_{timestamp}.json"
    with open(filename, 'w') as f:
        json.dump(history, f, indent=2)
    print(f"\nGame replay saved as: {filename}")

def get_difficulty_settings() -> Tuple[int, float]:
    """Get game settings based on difficulty level"""
    print(f"\n{Fore.YELLOW}Select Difficulty:{Style.RESET_ALL}")
    print(f"1. {Fore.GREEN}Easy{Style.RESET_ALL} (80% target)")
    print(f"2. {Fore.YELLOW}Medium{Style.RESET_ALL} (70% target)")
    print(f"3. {Fore.RED}Hard{Style.RESET_ALL} (60% target)")
    
    while True:
        choice = input("Choose difficulty (1-3): ")
        if choice == "1":
            return 5, 0.8  # rounds, target percentage
        elif choice == "2":
            return 7, 0.7
        elif choice == "3":
            return 10, 0.6
        print(f"{Fore.RED}Invalid choice. Try again.{Style.RESET_ALL}")

def main():
    play_sound("welcome")
    welcome = "Welcome to Checkmate - Enhanced Edition"
    delay = 0.1
    for char in welcome:
        sys.stdout.write(f"{Fore.GREEN}{char}{Style.RESET_ALL}{Fore.RED}_{Style.RESET_ALL}")
        sys.stdout.flush()
        time.sleep(delay)
        
    while True:
        print(f"\n1.{Fore.BLUE} Start{Style.RESET_ALL}")
        print(f"2.{Fore.YELLOW} Rules{Style.RESET_ALL}")
        print(f"3.{Fore.RED} Exit{Style.RESET_ALL}")
        choice = input("Choose an option: ")
        
        if choice == "1":
            rounds, target_percent = get_difficulty_settings()
            start(rounds, target_percent)
        elif choice == "2":
            display_rules()
        elif choice == "3":
            print(f"{Fore.YELLOW}Thanks for Playing!{Style.RESET_ALL}")
            break
        else:
            print(f"{Fore.RED}Try Again{Style.RESET_ALL}")
            
def display_rules():
    rules = f"""
    {Fore.CYAN}Game Rules:{Style.RESET_ALL}
    1. Multiple players compete in rounds
    2. Each player selects a number between 0 and 100
    3. Target is calculated as percentage of average (varies by difficulty)
    4. Closest player to target wins the round
    5. Other players get -1 point
    6. Players are eliminated at -5 points
    7. Last player standing wins!
    
    {Fore.YELLOW}Difficulty Levels:{Style.RESET_ALL}
    - Easy: 5 rounds, target = 80% of average
    - Medium: 7 rounds, target = 70% of average
    - Hard: 10 rounds, target = 60% of average
    """
    print(rules)
            
def get_valid_input(prompt: str) -> int:
    while True:
        try:
            user_input = int(input(prompt))
            if 0 <= user_input <= 100:
                return user_input
            print(f"{Fore.YELLOW}Please enter a number between 0 and 100.{Style.RESET_ALL}")
        except ValueError:
            print(f"{Fore.RED}Invalid input. Please enter a valid number.{Style.RESET_ALL}")
    
def start(max_rounds: int, target_percent: float):
    game_history = []
    result = [0, 0, 0, 0, 0]
    player_no_list = []
    players_name_list = []
    
    # Player registration
    print(f"\n{Fore.GREEN}Player Registration:{Style.RESET_ALL}")
    for pn in range(1, 6):
        print(f"Enter Player{pn} name: ")
        name = input()
        player_no_list.append(f"p{pn}")
        players_name_list.append(name)
    
    # Display initial player list
    display_players(player_no_list, players_name_list)
    
    for round_num in range(1, max_rounds + 1):
        print(f"\n{Fore.GREEN}Round {round_num} of {max_rounds}{Style.RESET_ALL}")
        ans = []
        
        # Collect player answers
        for no, name in zip(player_no_list, players_name_list):
            print(f"\n{Fore.CYAN}{name}'s turn:{Style.RESET_ALL}")
            an = get_valid_input(f"{no}> Enter a number between 0 and 100: ")
            ans.append(int(an))
            
        # Calculate round results
        total = sum(ans)
        av = total/len(ans)
        target = av * target_percent
        print(f"\n{Fore.YELLOW}Round Results:{Style.RESET_ALL}")
        print(f"Numbers played: {ans}")
        print(f"Total: {total}")
        print(f"Average: {av:.2f}")
        print(f"Target ({int(target_percent * 100)}% of average): {target:.2f}")
        
        # Calculate differences and find winner
        dif = [abs(target-i) for i in ans]
        print(f"Differences from target: {[f'{d:.2f}' for d in dif]}")
        
        minn = min(dif)
        min_index = dif.index(minn)
        closest_no = ans[min_index]
        
        # Update scores
        marks = [0 if i == closest_no else -1 for i in ans]
        for i in range(len(ans)):
            result[i] = result[i] + marks[i]
            
        # Record round history
        round_data = {
            "round": round_num,
            "numbers": ans,
            "target": target,
            "winner": players_name_list[min_index],
            "scores": result.copy()
        }
        game_history.append(round_data)
            
        # Check for winner
        if len(result) == 1:
            announce_winner(players_name_list[0])
            save_game_history(game_history)
            break
            
        # Check for eliminations
        eliminated = check_eliminations(result, player_no_list, players_name_list)
        if eliminated:
            play_sound("eliminate")
            
        # Display current standings
        display_standings(player_no_list, players_name_list, result)
        
        if len(result) == 1:
            announce_winner(players_name_list[0])
            save_game_history(game_history)
            break
            
def display_players(player_nos: List[str], player_names: List[str]):
    print("\nCurrent Players:")
    for pno, pname in zip(player_nos, player_names):
        print(f"{Fore.CYAN}{pno}-{pname}{Style.RESET_ALL}")
        
def display_standings(player_nos: List[str], player_names: List[str], scores: List[int]):
    print(f"\n{Fore.YELLOW}Current Standings:{Style.RESET_ALL}")
    for pno, pname, score in zip(player_nos, player_names, scores):
        color = Fore.GREEN if score >= 0 else Fore.RED
        print(f"{color}{pno}-{pname}: {score} points{Style.RESET_ALL}")
        
def check_eliminations(result: List[int], player_nos: List[str], 
                      player_names: List[str]) -> bool:
    eliminated = False
    i = 0
    while i < len(result):
        if result[i] <= -5:
            print(f"\n{Fore.RED}{player_names[i]} has been eliminated!{Style.RESET_ALL}")
            result.pop(i)
            player_nos.pop(i)
            player_names.pop(i)
            eliminated = True
        else:
            i += 1
    return eliminated
    
def announce_winner(winner_name: str):
    play_sound("winner")
    print(f"\n{Fore.GREEN}🎉 Congratulations! {winner_name} is the Champion! 🎉{Style.RESET_ALL}")

if __name__ == "__main__":
    main()